# Azure Responses Client Direct Usage Example

This notebook demonstrates direct `AzureResponsesClient` usage for structured response generation with Azure OpenAI models.

**Features:**
- Function calling capabilities with custom business logic
- Structured output generation with Pydantic models
- Streaming and non-streaming response modes

**Prerequisites:**
- Run `az login` in your terminal for Azure CLI authentication
- Ensure your `.env` file has proper Azure OpenAI configuration

In [ ]:
# Copyright (c) Microsoft. All rights reserved.

# Import Required Libraries
import os
from random import randint
from typing import Annotated

from agent_framework.exceptions import ServiceResponseException
from agent_framework.openai import OpenAIResponsesClient
from azure.identity import AzureCliCredential
from dotenv import load_dotenv
from pydantic import BaseModel, Field

load_dotenv(override=True)

REQUIRED_ENV_VARS = ["AZURE_OPENAI_ENDPOINT"]
missing_env_vars = [var for var in REQUIRED_ENV_VARS if not os.environ.get(var)]
if not os.environ.get("AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME") and not os.environ.get(
    "AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"
):
    missing_env_vars.append(
        "AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME or AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"
    )
if missing_env_vars:
    raise OSError("Missing required environment variables: " + ", ".join(missing_env_vars))

AZURE_OPENAI_ENDPOINT = os.environ["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_DEPLOYMENT = (
    os.environ.get("AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME")
    or os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"]
)


def build_responses_client() -> OpenAIResponsesClient:
    """Create an Azure OpenAI Responses client using environment configuration."""
    return OpenAIResponsesClient(
        credential=AzureCliCredential(),
        endpoint=AZURE_OPENAI_ENDPOINT,
        deployment_name="gpt-5",
    )


def handle_service_exception(context: str, exc: ServiceResponseException) -> None:
    """Provide actionable guidance when Azure OpenAI returns an error."""
    error_text = str(exc)
    print(f"{context} failed: {error_text}")
    if "404" in error_text:
        print(
            "Hint: Verify that the deployment name exists in your Azure OpenAI resource "
            "and that AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME (or chat fallback) matches it."
        )
    elif "401" in error_text:
        print(
            "Hint: Authenticate with `az login` or provide a valid "
            "AZURE_OPENAI_API_KEY / Azure credential."
        )
    elif "429" in error_text:
        print(
            "Hint: The resource is rate limited. Consider reducing requests "
            "or upgrading your quota."
        )


client = build_responses_client()

## Define Custom Tools

Create a simple weather tool that the agent can call to get weather information for different locations.

In [ ]:
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return (
        f"The weather in {location} is {conditions[randint(0, 3)]} "
        f"with a high of {randint(10, 30)}°C."
    )

## Define Structured Output Format

Use Pydantic models to define the expected structure of the agent's response.

In [ ]:
class OutputStruct(BaseModel):
    """Structured output for weather information."""

    location: str
    weather: str

## Initialize the Client and Get Response

Create an Azure OpenAI Responses Client and query for weather information with structured output.

In [ ]:
# Display the configuration being used for clarity
print(f"Configured endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"Configured deployment: {AZURE_OPENAI_DEPLOYMENT}")


# Define the user message
message = "What's the weather in Amsterdam and in Paris?"
print(f"User: {message}")

## Option 1: Streaming Response

Get a streaming response from the model, which provides real-time output as it's generated.

In [ ]:
# Ensure your .env file contains AZURE_OPENAI_ENDPOINT and either
# AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME or
# AZURE_OPENAI_CHAT_DEPLOYMENT_NAME with correct values
try:
    response: OpenAIResponsesClient = await client.create(
        messages=[{"role": "user", "content": message}],
    )
    print(f"Assistant: {response.content}")
except ServiceResponseException as exc:
    handle_service_exception("Non-streaming creation", exc)

## Option 2: Non-Streaming Response

Get a complete response in a single call without streaming.

In [ ]:
# Get non-streaming response
try:
    response = await client.get_response(message, tools=get_weather, response_format=OutputStruct)
    print(f"Assistant: {response.value}")
except ServiceResponseException as exc:
    handle_service_exception("Non-streaming response", exc)